# レポート2

## Requirements
- python3系
- MeCab 
- numpy, gensim, mecabのpythonバインディング
- このノートブックと同じ階層にdataディレクトリを作成し, [こちら](https://www.rondhuit.com/download.html#ldcc) からデータをダウンロードして配置
- [こちら](http://aial.shiroyagi.co.jp/2017/02/japanese-word2vec-model-builder/) からword2vecモデルを取得して配置

## ディレクトリ構成

```
.
├── data
│   ├── topic-news-5903225.txt
│   │    ...
│   └── topic-news-6918105.txt
├── latest-ja-word2vec-gensim-model
│   ├── word2vec.gensim.model
│   ├── word2vec.gensim.model.syn1neg.npy
│   └── word2vec.gensim.model.wv.syn0.npy
└── report2.ipynb
```

In [11]:
from pathlib import Pathb
import numpy as np
from gensim.models.word2vec import Word2Vec
import MeCab

In [3]:
model = Word2Vec.load('./latest-ja-word2vec-gensim-model/word2vec.gensim.model')

In [12]:
class Tokenizer:
    @classmethod
    def tokenize(cls, sentence, use_neologd=True, dic_path='/usr/lib/mecab/dic/mecab-ipadic-neologd'):
        if use_neologd:
            option = f'-O wakati -d {dic_path}'
        else:
            option = '-O wakati'
        
        t = MeCab.Tagger(option)
        
        parse_result = t.parse(sentence).replace(' \n', '')
        tokens = parse_result.split(' ')
        return tokens
    
    @classmethod
    def tokenize_and_filter(cls, sentence, use_neologd=True, dic_path='/usr/lib/mecab/dic/mecab-ipadic-neologd', filter_by=["名詞", "動詞", "形容詞", "形容動詞"]):
        if use_neologd:
            option = f'-O chasen -d {dic_path}'
        else:
            option = '-O chasen'
        t = MeCab.Tagger(option)
        
        tokens = []
        
        parse_result = t.parse(sentence)
        morphemes = parse_result.split('\n')[:-2]
        for morph in morphemes:
            surf, _, _, pos, _, _ = morph.split('\t')
            if pos.split('-')[0] in filter_by:
                tokens.append(surf)
        return tokens

In [84]:
# データ読み込み

def split_line(line):
    """各行を文章に分割する. その際, かぎかっこ中の。では分割しないように注意する"""
    sentences = []
    
    in_quote_count = 0
    tmp = []
    for char in line:
        if char == '。' and in_quote_count == 0:
            tmp.append(char)
            sentences.append(''.join(tmp))
            tmp = []
        elif char == '「' or char == '『':
            in_quote_count += 1
            tmp.append(char)
        elif char == '」' or char == '』':
            in_quote_count -= 1
            tmp.append(char)
        else:
            tmp.append(char)
    return sentences
        

def preprocess(lines):
    """given lines of text, return list of (list of token)"""
    token_seqs = []
    sentences = []
    for line in lines:
        line = line.replace('\n', '')
        
        for sen in split_line(line):

            # 関連記事 or 関連情報以下は無視
            if sen == '【関連情報】' or sen == '【関連記事】':
                break
            # 行が空でないかチェック
            if sen:
                token_seq = Tokenizer.tokenize_and_filter(sen, use_neologd=False)
                token_seqs.append(token_seq)
                sentences.append(sen)
    return token_seqs, sentences


def calc_score(token_seq, target_word, min_word=6):
    score = 0
    ignore_count = 0
    for token in token_seq:
        if token not in model.wv:
            ignore_count += 1
            continue
        score += model.wv.similarity(token, target_word)
    num_token = len(token_seq) - ignore_count
    if num_token >= min_word:
        return score / (len(token_seq) - ignore_count)
    else:
        return 0

In [85]:
target_word = '怒り'

token_seqs = []
sentences = []

for p in Path('./data').glob('*.txt'):
    with p.open('r') as f:
        lines = f.readlines()
        
        token_seqs_tmp, sentences_tmp = preprocess(lines[3:])
        token_seqs.extend(token_seqs_tmp)
        sentences.extend(sentences_tmp)
        
# 各文章の〇〇度合を測る
result = []
for token_seq, sentence in zip(token_seqs, sentences):
    result.append({
        'sentence': sentence,
        'token_seq': token_seq,
        'score': calc_score(token_seq, target_word, min_word=1)
    })

# scoreの降順にソート
sorted(result, key=lambda k: k['score'], reverse=True)[:10]

[{'sentence': '本当に驚きました。', 'token_seq': ['驚き'], 'score': 0.773547554150178},
 {'sentence': 'でも不安だ。', 'token_seq': ['不安'], 'score': 0.6254362622572959},
 {'sentence': 'と大変お怒りの様子。',
  'token_seq': ['大変', '怒り', '様子'],
  'score': 0.6157276879175946},
 {'sentence': 'たまらないね。', 'token_seq': ['たまらない'], 'score': 0.5464817954724311},
 {'sentence': 'すごく、謝りました。',
  'token_seq': ['すごく', '謝り'],
  'score': 0.5382587800835938},
 {'sentence': '恥を知ってほしいね。',
  'token_seq': ['恥', '知っ', 'ほしい'],
  'score': 0.5379403460838145},
 {'sentence': '不安だらけだ。',
  'token_seq': ['不安', 'だらけ'],
  'score': 0.5346223609270333},
 {'sentence': '正直に、はっきりと聞きます。',
  'token_seq': ['正直', '聞き'],
  'score': 0.531903227676507},
 {'sentence': 'その顛末はいかに——。',
  'token_seq': ['顛末', '——。'],
  'score': 0.5109663268328465},
 {'sentence': '私もごめんなさいって言いますし。',
  'token_seq': ['私', '言い'],
  'score': 0.5069577043037405}]

In [86]:
target_word = '怒り'

token_seqs = []
sentences = []

for p in Path('./data').glob('*.txt'):
    with p.open('r') as f:
        lines = f.readlines()
        
        token_seqs_tmp, sentences_tmp = preprocess(lines[3:])
        token_seqs.extend(token_seqs_tmp)
        sentences.extend(sentences_tmp)
        
# 各文章の〇〇度合を測る
result = []
for token_seq, sentence in zip(token_seqs, sentences):
    result.append({
        'sentence': sentence,
        'token_seq': token_seq,
        'score': calc_score(token_seq, target_word, min_word=6)
    })

# scoreの降順にソート
sorted(result, key=lambda k: k['score'], reverse=True)[:10]

[{'sentence': '思ったこと『あのブスめ！』と、怒り心頭だ。',
  'token_seq': ['思っ', 'こと', 'ブス', 'め', '怒り', '心頭'],
  'score': 0.4767247590137995},
 {'sentence': '菜々緒が杉村太蔵のセクハラ発言に嫌悪の表情。',
  'token_seq': ['菜', '緒', '杉村', '太蔵', 'セクハラ', '発言', '嫌悪', '表情'],
  'score': 0.4156870290483231},
 {'sentence': 'のメンバーは笑いつつも「ひどい」「びっくり」とさんまの発言に異議を唱えた。',
  'token_seq': ['メンバー', '笑い', 'ひどい', 'びっくり', 'さんま', '発言', '異議', '唱え'],
  'score': 0.41488492371010827},
 {'sentence': 'まあ、ファンの方に迷惑をかけてしまったので、心配かけてしまったので、申し訳ないなって。',
  'token_seq': ['ファン', '方', '迷惑', 'かけ', 'しまっ', '心配', 'かけ', 'しまっ', '申し訳'],
  'score': 0.41136217118679524},
 {'sentence': '私たちはGhaddafi（カダフィ）のことを忘れない、安らかに。',
  'token_seq': ['私', 'たち', 'Ghaddafi', 'カダフィ', 'こと', '忘れ', '安らか'],
  'score': 0.408436657026593},
 {'sentence': 'すると、店屋のおじさんが、「おい、ガキ、ちょっと待てや。」とかなり乱暴な口調で私を引き留めました。',
  'token_seq': ['店屋', 'おじさん', 'ガキ', '待て', '乱暴', '口調', '私', '引き留め'],
  'score': 0.39957401628787415},
 {'sentence': 'マツコは即座に「今のセリフちょうだい!」と褒めるくらい、この言葉を気に入ったという。',
  'token_seq': ['マツコ',
   '即座',
   '今

In [92]:
target_word = '悲しい'

token_seqs = []
sentences = []

for p in Path('./data').glob('*.txt'):
    with p.open('r') as f:
        lines = f.readlines()
        
        token_seqs_tmp, sentences_tmp = preprocess(lines[3:])
        token_seqs.extend(token_seqs_tmp)
        sentences.extend(sentences_tmp)
        
# 各文章の〇〇度合を測る
result = []
for token_seq, sentence in zip(token_seqs, sentences):
    result.append({
        'sentence': sentence,
        'token_seq': token_seq,
        'score': calc_score(token_seq, target_word, min_word=1)
    })

# scoreの降順にソート
sorted(result, key=lambda k: k['score'], reverse=True)[:10]

[{'sentence': 'たまらないね。', 'token_seq': ['たまらない'], 'score': 0.7413013005825441},
 {'sentence': '本当に驚きました。', 'token_seq': ['驚き'], 'score': 0.6876717736134687},
 {'sentence': 'すごく、謝りました。',
  'token_seq': ['すごく', '謝り'],
  'score': 0.6828894231943108},
 {'sentence': 'と冗談で締め括った。',
  'token_seq': ['冗談', '締め括っ'],
  'score': 0.600302032664652},
 {'sentence': '私もごめんなさいって言いますし。',
  'token_seq': ['私', '言い'],
  'score': 0.5780926845270912},
 {'sentence': 'やっちゃっていいです。',
  'token_seq': ['やっ', 'ちゃっ', 'いい'],
  'score': 0.5775472536920402},
 {'sentence': '明石家さんまが心境を語った。',
  'token_seq': ['明石家', 'さんま', '心境', '語っ'],
  'score': 0.567050635950566},
 {'sentence': '明石家さんまが心境を語った。',
  'token_seq': ['明石家', 'さんま', '心境', '語っ'],
  'score': 0.567050635950566},
 {'sentence': '正直に、はっきりと聞きます。',
  'token_seq': ['正直', '聞き'],
  'score': 0.5646038056939344},
 {'sentence': '「おい、誰だ笑ってるの！」。',
  'token_seq': ['誰', '笑っ', 'てる'],
  'score': 0.5641643368358754}]

In [93]:
target_word = '悲しい'

token_seqs = []
sentences = []

for p in Path('./data').glob('*.txt'):
    with p.open('r') as f:
        lines = f.readlines()
        
        token_seqs_tmp, sentences_tmp = preprocess(lines[3:])
        token_seqs.extend(token_seqs_tmp)
        sentences.extend(sentences_tmp)
        
# 各文章の〇〇度合を測る
result = []
for token_seq, sentence in zip(token_seqs, sentences):
    result.append({
        'sentence': sentence,
        'token_seq': token_seq,
        'score': calc_score(token_seq, target_word, min_word=6)
    })

# scoreの降順にソート
sorted(result, key=lambda k: k['score'], reverse=True)[:10]

[{'sentence': 'すると、店屋のおじさんが、「おい、ガキ、ちょっと待てや。」とかなり乱暴な口調で私を引き留めました。',
  'token_seq': ['店屋', 'おじさん', 'ガキ', '待て', '乱暴', '口調', '私', '引き留め'],
  'score': 0.4824519317820455},
 {'sentence': 'マツコは即座に「今のセリフちょうだい!」と褒めるくらい、この言葉を気に入ったという。',
  'token_seq': ['マツコ',
   '即座',
   '今',
   'セリフ',
   'ちょうだい',
   '!」',
   '褒める',
   '言葉',
   '気に入っ',
   'いう'],
  'score': 0.48187331631335023},
 {'sentence': 'CMやってるから）こっちもこっちでコマーシャルやってるんで。',
  'token_seq': ['やっ', 'てる', 'こっち', 'こっち', 'コマーシャル', 'やっ', 'てる'],
  'score': 0.47590738270378635},
 {'sentence': '簡単に言うと、会いたくて会いたくて震える曲です。',
  'token_seq': ['簡単', '言う', '会い', '会い', '震える', '曲'],
  'score': 0.43042770075496656},
 {'sentence': '私は辞めますと言って、秋元先生は辞めなさいと。',
  'token_seq': ['私', '辞め', '言っ', '秋元', '先生', '辞め', 'なさい'],
  'score': 0.4275552148569907},
 {'sentence': 'さらに、SMAPの中居正広に「カズさんと一緒にやったら?」と振られると「カズ！って言ってみたいです」と笑顔で返した。',
  'token_seq': ['SMAP',
   '中居',
   '正広',
   'カズ',
   'さん',
   '一緒',
   'やっ',
   '?」',
   '振ら',
   'れる',
   'カズ',
   '言っ',
   'みたい',
   '笑顔',
   '返

In [39]:
sorted(token_seqs, key=lambda k: k['score'], reverse=True)

[{'score': 0.7122926801481088, 'token_seq': ['お前', '言う']},
 {'score': 0.7102390935864061, 'token_seq': ['やっ', 'ちゃっ', 'いい']},
 {'score': 0.6799907018147473, 'token_seq': ['お前', 'つまらない']},
 {'score': 0.6307879347704235, 'token_seq': ['やり']},
 {'score': 0.5945888680821403, 'token_seq': ['プライベート', 'メイサ', '支えれ', 'いい']},
 {'score': 0.5548862992191734, 'token_seq': ['やっ', 'ください']},
 {'score': 0.5532956688805051, 'token_seq': ['中略']},
 {'score': 0.5532956688805051, 'token_seq': ['中略']},
 {'score': 0.5527532741079644,
  'token_seq': ['私', '捨てる', '言っ', 'ん', '旦那', 'さん', '食べる', '言い張っ']},
 {'score': 0.5521035202338793, 'token_seq': ['ためし']},
 {'score': 0.541222265252075, 'token_seq': ['何', '人', 'やっ', 'ちゃっ', 'いい']},
 {'score': 0.5346579323959039, 'token_seq': ['本人', '言い']},
 {'score': 0.5133843321454125,
  'token_seq': ['絵', 'すごい', 'うまかっ', '頑張っ', '欲しい']},
 {'score': 0.49976563999205265, 'token_seq': ['嫌', 'やめろ']},
 {'score': 0.4977297886714571, 'token_seq': ['奴', '好き', 'ん', 'しょうが']},
 {'score': 0.49

In [20]:
model.wv.similar_by_word('嬉しい')

[('うれしい', 0.9171843528747559),
 ('嬉しかっ', 0.8691217303276062),
 ('すごい', 0.8576909899711609),
 ('やれる', 0.8458874821662903),
 ('ありがたい', 0.8440902829170227),
 ('楽しかっ', 0.8345097899436951),
 ('恥ずかしい', 0.8334256410598755),
 ('申し訳ない', 0.8254172205924988),
 ('すごく', 0.8196461200714111),
 ('皆さん', 0.8161286115646362)]

In [22]:
model.wv.similar_by_word('いい')

[('うまい', 0.793982982635498),
 ('言う', 0.7922905683517456),
 ('ありがたい', 0.7680160403251648),
 ('言い', 0.7668641805648804),
 ('くくる', 0.763872504234314),
 ('困る', 0.7616062164306641),
 ('云う', 0.7576260566711426),
 ('なあ', 0.7515909075737),
 ('いって', 0.7459474205970764),
 ('言い放つ', 0.732398509979248)]

In [24]:
model.wv.similar_by_word('厳しい')

[('厳しく', 0.7486790418624878),
 ('不安', 0.7111629247665405),
 ('悪さ', 0.7081772685050964),
 ('極度', 0.678291380405426),
 ('それなり', 0.65279221534729),
 ('厳しかっ', 0.648469865322113),
 ('重い', 0.6451793909072876),
 ('苦しい', 0.6360158920288086),
 ('悪い', 0.635636568069458),
 ('風紀', 0.6352398991584778)]